In [151]:
import numpy as np
from scipy import integrate

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation

N_trajectories = 20

In [158]:
def chua_deriv((x, y, z), t0, alpha=9.5, beta=14.):
    """Compute the time-derivative of a chua system.""" 
    f = ((1./16.)*x**3 - (1./6.)*x)
    return [alpha * (y  - f),  x - y + z, -(beta * y)]

In [159]:
#def lorentz_deriv((x, y, z), t0, sigma=10., beta=8./3, rho=28.0):
#    """Compute the time-derivative of a Lorentz system."""
#    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

In [156]:
# Choose random starting points, uniformly distributed from -15 to 15
np.random.seed(1)
x0 =  .001 * np.random.random((N_trajectories, 3))
print x0
# Solve for the trajectories
t = np.linspace(0, 100, 10000)
x_t = np.asarray([integrate.odeint(chua_deriv, x0i, t)
                  for x0i in x0])

# Set up figure & 3D axis for animation
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1], projection='3d')
ax.axis('off')

# choose a different color for each trajectory
colors = plt.cm.jet(np.linspace(0, 1, N_trajectories))

# set up lines and points
lines = sum([ax.plot([], [], [], '-', c=c)
             for c in colors], [])
pts = sum([ax.plot([], [], [], 'o', c=c)
           for c in colors], [])

# prepare the axes limits
ax.set_xlim((-2, 2))
ax.set_ylim((-2, 2))
ax.set_zlim((-2, 2))

# set point-of-view: specified by (altitude degrees, azimuth degrees)
ax.view_init(30, 0)

[[  4.17022005e-04   7.20324493e-04   1.14374817e-07]
 [  3.02332573e-04   1.46755891e-04   9.23385948e-05]
 [  1.86260211e-04   3.45560727e-04   3.96767474e-04]
 [  5.38816734e-04   4.19194514e-04   6.85219500e-04]
 [  2.04452250e-04   8.78117436e-04   2.73875932e-05]
 [  6.70467510e-04   4.17304802e-04   5.58689828e-04]
 [  1.40386939e-04   1.98101489e-04   8.00744569e-04]
 [  9.68261576e-04   3.13424178e-04   6.92322616e-04]
 [  8.76389152e-04   8.94606664e-04   8.50442114e-05]
 [  3.90547832e-05   1.69830420e-04   8.78142503e-04]
 [  9.83468338e-05   4.21107625e-04   9.57889530e-04]
 [  5.33165285e-04   6.91877114e-04   3.15515631e-04]
 [  6.86500928e-04   8.34625672e-04   1.82882773e-05]
 [  7.50144315e-04   9.88861089e-04   7.48165654e-04]
 [  2.80443992e-04   7.89279328e-04   1.03226007e-04]
 [  4.47893526e-04   9.08595503e-04   2.93614148e-04]
 [  2.87775339e-04   1.30028572e-04   1.93669579e-05]
 [  6.78835533e-04   2.11628116e-04   2.65546659e-04]
 [  4.91573159e-04   5.33625

In [157]:
# initialization function: plot the background of each frame
def init():
    for line, pt in zip(lines, pts):
        line.set_data([], [])
        line.set_3d_properties([])

        pt.set_data([], [])
        pt.set_3d_properties([])
    return lines + pts

# animation function.  This will be called sequentially with the frame number
def animate(i):
    # we'll step two time-steps per frame.  This leads to nice results.
    i = (2 * i) % x_t.shape[1]

    for line, pt, xi in zip(lines, pts, x_t):
        x, y, z = xi[:i].T
        line.set_data(x, y)
        line.set_3d_properties(z)

        pt.set_data(x[-1:], y[-1:])
        pt.set_3d_properties(z[-1:])

    ax.view_init(30, 0.3 * i)
    fig.canvas.draw()
    return lines + pts

# instantiate the animator.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=5000, interval=30, blit=True)

# Save as mp4. This requires mplayer or ffmpeg to be installed
#anim.save('lorentz_attractor.mp4', fps=15, extra_args=['-vcodec', 'libx264'])

plt.show()